In [13]:
import sys
sys.path.append('/Users/liangwang/GitHub/DataAnalysis')

In [14]:
import numpy as np
import os
from pathlib import Path
import pathlib
import platform
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
from scipy import signal 
from scipy import odr 
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from statsmodels.regression.rolling import RollingOLS

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [15]:
pathlib.WindowsPath = pathlib.PosixPath

In [16]:
# biolinq_col_measurement = ['biolinq_no_blank','biolinq_s1','biolinq_s3','biolinq_s4']
# biolinq_col_keep = ['ms_since_boot','algo_state','algo_state_desc','rate_of_change','is_rate_of_change_valid',
#                     'time_from_start','biolinq_s1','biolinq_s3','biolinq_s4','biolinq_no_blank','sensor_id','UTC_Time',
#                     'color_code','color_desc']
reference_setting = {
    'ysi': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
        'value_lb': 0,
        'value_ub': 900
    },
    'acck': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
        'value_lb': 10,
        'value_ub': 600
    },
    'Libre': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
        'value_lb': 40,
        'value_ub': 400
    },
    'Dexcom': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
        'value_lb': 40,
        'value_ub': 400
    }         
}

In [20]:
data_root_path = '/Volumes/Shared/Algo/Algorithm Output/Isabella_v02 time varying trained by ysi'
study_folder = ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','Pre-Piv-2-training','iBlinqAcet','iBlinqFW231','iBlinqOverlay']
save_agg_biolinq_ref_df = True
save_agg_biolinq_df = True
folder_to_save = '/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Gen 1/Algorithm Development/Gen 1 Model Comparison'
agg_biolinq_ref_df_file_name = 'agg_biolinq_ref_df.csv'
agg_biolinq_df_file_name = 'agg_biolinq_df.csv'
agg_biolinq_ref_df_file_path = os.path.join(folder_to_save,agg_biolinq_ref_df_file_name)
agg_biolinq_df_file_path = os.path.join(folder_to_save,agg_biolinq_df_file_name)

In [21]:
head_agg_biolinq_ref_df = False
head_agg_biolinq_df = False
for study in study_folder:
    study_full_path = os.path.join(data_root_path,study)
    loaded_data = FileRead.load_pickle('alg_out.zip', study_full_path, as_dict=True)
    all_sensor_data = loaded_data['all_sensor_data']
    sensor_map_df = loaded_data['sensor_map_df'][['SN', 'Device Observations', 'Chemistry', 'Subject ID', 'Location', 'Grouping']]
    for sensor in all_sensor_data.keys():
        print(f'Processing sensor: {sensor}')
        current_sensor_data = all_sensor_data[sensor].copy()
        biolinq_exist = False
        ref_exist = {}
        ref_exist['acck_exist'] = False
        ref_exist['ysi_exist'] = False
        ref_exist['Libre_exist'] = False
        ref_exist['Dexcom_exist'] = False
        ref_df_dict = {}
        biolinq_ref_df_dict = {}
        if 'biolinq' in current_sensor_data.keys():
            biolinq_exist = True
            biolinq_df_unmerge = current_sensor_data['biolinq'].copy()
            if biolinq_df_unmerge.empty:
                biolinq_exist = False
        
        if not biolinq_exist:
            continue
        biolinq_df = biolinq_df_unmerge.merge(sensor_map_df,how='left',left_on='sensor_id',right_on='SN') 
        biolinq_df['Study'] = study
        
        for this_ref_name,this_ref_setting in reference_setting.items():
            if this_ref_name in current_sensor_data.keys():
                ref_exist[f'{this_ref_name}_exist'] = True
                ref_df_dict[f'{this_ref_name}_df'] = current_sensor_data[this_ref_name].copy()
                if ref_df_dict[f'{this_ref_name}_df'].empty:
                    ref_exist[f'{this_ref_name}_exist'] = False
                else:
                    ref_df_dict[f'{this_ref_name}_df'].rename(columns={this_ref_name:'ref_value'},inplace=True)
                    ref_df_dict[f'{this_ref_name}_df']['ref_type'] = this_ref_name
                    valid_idx = ref_df_dict[f'{this_ref_name}_df']['ref_value'].ge(this_ref_setting['value_lb']) \
                        & ref_df_dict[f'{this_ref_name}_df']['ref_value'].le(this_ref_setting['value_ub'])
                    ref_df_dict[f'{this_ref_name}_df'] = ref_df_dict[f'{this_ref_name}_df'].loc[valid_idx,:]
            if not ref_exist[f'{this_ref_name}_exist']:
                continue
            time_to_match = this_ref_setting['time_to_match']
            direction_to_match = this_ref_setting['direction_to_match']
            tol_to_match = this_ref_setting['tol_to_match']
            biolinq_ref_df_dict[f'biolinq_{this_ref_name}_df'] = pd.merge_asof(left=ref_df_dict[f'{this_ref_name}_df'].dropna(subset=[time_to_match]).sort_values(by=time_to_match), right=biolinq_df.dropna(subset=[time_to_match]).sort_values(by=time_to_match),
                                        on=time_to_match, direction=direction_to_match, tolerance=pd.Timedelta(tol_to_match))

            #Save to CSV
            if save_agg_biolinq_ref_df:
                if head_agg_biolinq_ref_df==False:
                    biolinq_ref_df_dict[f'biolinq_{this_ref_name}_df'].to_csv(agg_biolinq_ref_df_file_path,mode='a',float_format='%g',index=False,header=True)
                    head_agg_biolinq_ref_df=True
                else:
                    biolinq_ref_df_dict[f'biolinq_{this_ref_name}_df'].to_csv(agg_biolinq_ref_df_file_path,mode='a',float_format='%g',index=False,header=False) 

            if save_agg_biolinq_df:
                if head_agg_biolinq_df==False:
                    biolinq_df.to_csv(agg_biolinq_df_file_path,mode='a',float_format='%g',index=False,header=True)
                    head_agg_biolinq_df=True
                else:
                    biolinq_df.to_csv(agg_biolinq_df_file_path,mode='a',float_format='%g',index=False,header=False)                     

          